<a href="https://colab.research.google.com/github/nathanfrib/tcc_rnas_2024/blob/main/otimizacao_hiperparametros_mlp_lstm_gru_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import ParameterSampler
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Conv1D, MaxPooling1D, Flatten
from keras.losses import MeanSquaredError
from keras.optimizers import Adam
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

# vento_treino: vetor com todas amostras normalizadas de velocidade do vento de 2021 e 2022
# vento_teste: vetor com todas amostras normalizadas de velocidade do vento de 2023

def df_to_X_y(df, window_size):
  df_as_numpy = df.to_numpy()
  X = []
  Y = []
  for i in range(len(df_as_numpy) - window_size):
    row = [[a] for a in df_as_numpy[i:i+window_size]]
    X.append(row)
    label = df_as_numpy[i+window_size]
    Y.append(label)
  return np.array(X), np.array(Y)

# Gerando dados fictícios de velocidade do vento
np.random.seed(42)

# Gerar 2 anos de dados de velocidade do vento (2021 e 2022)
n_samples_treino = int(52_560 * 2)  # Aproximadamente 105.120 amostras
vento_treino = np.random.uniform(low=0, high=20, size=n_samples_treino)  # Velocidade do vento entre 0 e 20 m/s

# Gerar 1 ano de dados de velocidade do vento (2023)
n_samples_teste = 52_560  # Aproximadamente 52.560 amostras
vento_teste = np.random.uniform(low=0, high=20, size=n_samples_teste)

lag = 18  # 3 horas para trás
X, y = df_to_X_y(pd.Series(vento_treino), lag)
X_teste, y_teste = df_to_X_y(pd.Series(vento_teste), lag)

# Funções para construção dos modelos via RandomSearchCV
def create_model(model_type, lag=lag, units=50, learning_rate=0.01, dropout_rate=0.0, filters=64, kernel_size=3):
    model = Sequential()
    if model_type == 'lstm':
        model.add(LSTM(units, activation='relu', input_shape=(lag, 1)))
    elif model_type == 'mlp':
        model.add(Dense(units, activation='relu', input_shape=(lag,)))
    elif model_type == 'gru':
        model.add(GRU(units, activation='relu', input_shape=(lag, 1)))
    elif model_type == 'cnn':
        model.add(Conv1D(filters, kernel_size, activation='relu', input_shape=(lag, 1)))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
    else:
        raise ValueError(f"Modelo não reconhecido: {model_type}")
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=learning_rate))
    return model

def train_and_evaluate(model_type, X_train, y_train, X_val, y_val, params):
    # Separar os hiperparâmetros usados na criação do modelo e no treinamento
    model_params = {k: v for k, v in params.items() if k not in ['batch_size']}
    training_params = {k: v for k, v in params.items() if k in ['batch_size']}

    model = create_model(model_type, **model_params)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    history = model.fit(X_train, y_train, epochs=50, batch_size=training_params['batch_size'],
                        validation_data=(X_val, y_val), verbose=1, callbacks=[early_stopping])
    y_pred = model.predict(X_val)
    score = r2_score(y_val, y_pred)
    return score

# Definição dos espaços de busca para os hiperparâmetros
param_dist_lstm = {
    'units': [50, 100, 200],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'dropout_rate': [0.0, 0.2, 0.4]
}

param_dist_mlp = {
    'units': [50, 100, 200],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'dropout_rate': [0.0, 0.2, 0.4]
}

param_dist_gru = {
    'units': [50, 100, 200],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'dropout_rate': [0.0, 0.2, 0.4]
}

param_dist_cnn = {
    'filters': [32, 64, 128],
    'kernel_size': [2, 3, 4],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'dropout_rate': [0.0, 0.2, 0.4]
}

# Função para executar a busca aleatória para cada modelo
def random_search_cv(model_type, param_dist, X, y, n_iter=10, cv=5):
    best_score = -np.inf
    best_params = None
    comb=0
    tscv = TimeSeriesSplit(n_splits=cv)
    for params in ParameterSampler(param_dist, n_iter=n_iter, random_state=42):
        fold_scores = []
        comb=comb+1

        for train_index, val_index in tscv.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            score = train_and_evaluate(model_type, X_train, y_train, X_val, y_val, params)
            fold_scores.append(score)

        mean_score = np.mean(fold_scores)
        print(f"Modelo: {model_type} | Combinação: {comb}")
        print(params)
        print(f"R2 média: {mean_score}")
        if mean_score > best_score:
            best_score = mean_score
            best_params = params

    return best_score, best_params

# Executar a busca aleatória para cada modelo
models = [
    ('mlp', param_dist_mlp),
    ('lstm', param_dist_lstm),
    ('gru', param_dist_gru),
    ('cnn', param_dist_cnn)
]

for model_type, param_dist in models:
    best_score, best_params = random_search_cv(model_type, param_dist, X, y)
    print(f"Melhor R2 para a arquitetura {model_type}: {best_score}")
    print(f"Melhores hiperparâmetros para a arquitetura {model_type}: {best_params}")